## Import Libraries

In [1]:
import os
import webbrowser
from neo4j import GraphDatabase
from pyvis.network import Network
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import time

## Load Data

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/movie-config.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Implement Constraints

### Goals

1. For the **Person** nodes:
   - `constraint_name` : **Person_tmdbId_unique**
   - `node_label` : **Person**
   - `property_key` : **tmdbId**

2. For the **User** nodes:
   - `constraint_name` : **User_userId_unique**
   - `node_label` : **User**
   - `property_key` : **userId**

3. For the **Genre** nodes:
   - `constraint_name` : **Genre_name_unique**
   - `node_label` : **Genre**
   - `property_key` : **name**

### Purpose:
- Ensure unique constraints for the key properties of each node type.
- Maintain data integrity in the graph database.
- Facilitate efficient queries and avoid duplication of entities.

In [5]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
def create_constraints(tx):
    tx.run("""
        CREATE CONSTRAINT Movie_movieId_unique IF NOT EXISTS FOR (x:Movie) REQUIRE x.movieId IS UNIQUE
    """)

    tx.run("""
        CREATE CONSTRAINT Person_tmdbId_unique IF NOT EXISTS FOR (p:Person) REQUIRE p.tmdbId IS UNIQUE
    """)

    tx.run("""
        CREATE CONSTRAINT User_userId_unique IF NOT EXISTS FOR (u:User) REQUIRE u.userId IS UNIQUE
    """)

    tx.run("""
        CREATE CONSTRAINT Genre_name_unique IF NOT EXISTS FOR (g:Genre) REQUIRE g.name IS UNIQUE
    """)


In [7]:
with driver.session() as session:
    session.execute_write(create_constraints)

In [8]:
# Define the queries
queries = [
    """
    MATCH (p:Person)
    WHERE p.name IS NOT NULL
    RETURN p, id(p)
    """,
    """
    MATCH (p:Person)
    WHERE p.name IS NULL
    SET p.name = "Unknown"
    """,
    """
    CREATE CONSTRAINT Movie_title_exists IF NOT EXISTS
    FOR (m:Movie)
    REQUIRE m.title IS NOT NULL
    """,
    """
    CREATE CONSTRAINT User_name_exists IF NOT EXISTS
    FOR (u:User)
    REQUIRE u.name IS NOT NULL
    """,
    """
    CREATE CONSTRAINT Person_name_exists IF NOT EXISTS
    FOR (x:Person)
    REQUIRE x.name IS NOT NULL
    """
]

In [ ]:
# Execute queries
with driver.session() as session:
    for query in queries:
        result = session.run(query)
        if result.peek():  # Only for queries with RETURN
            for record in result:
                print(record)